# Psychoinformatics - Week 11 (Exercises)
by your name (your email)

In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

from numpy import *
from matplotlib.pyplot import *
from IPython.display import *
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
from sklearn import *

iris = datasets.load_iris()
X=iris.data; Y=iris.target

## 1 Performance Tuning of a Neural Net (8 points)

### 1.0 Baseline Performance
SVM can reach an classifcation accuracy ~ 8x% correct for the HARD Iris problem.

In [4]:
sss=model_selection.StratifiedShuffleSplit(n_splits=3,test_size=0.1) # (45+45+45) vs. (5+5+5)
model=svm.SVC(C=10)
acc=[]
for train_index, test_index in sss.split(X, Y): # 3-fold cross-validation
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    model.fit(X_train[:,0:2],Y_train) #training
    acc.append(model.predict(X_test[:,0:2])==Y_test) # testing
print(np.mean(acc))

0.8


### 1.1 Tuning your ANN (4 points)
Tune your model hyperparameters (# of layers, # of units in each layer, activation function, optimizer, epochs, batch_size, etc.) to see if you can push your ANN performance up to ~9x% correct for the HARD iris problem.

In [7]:
from keras.models import Sequential,clone_model
from keras.layers import Dense
from keras.utils import to_categorical

model = Sequential()
model.add(Dense(units=3, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

acc=[]
for train_index, test_index in sss.split(X, Y): # 3-fold cross-validation
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    new_model=clone_model(model) # Otherwise the old model will keep learning
    new_model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
    new_model.fit(X_train[:,0:2], to_categorical(Y_train), epochs=50, batch_size=15)
    acc.append(np.mean(argmax(new_model.predict(X_test[:,0:2]),1)==Y_test)) # testing
print(acc, np.mean(acc))

Epoch 1/50
9/9 [==============================] - 0s 2ms/step - loss: 4.7487 - accuracy: 0.3333
Epoch 2/50
9/9 [==============================] - 0s 2ms/step - loss: 1.8556 - accuracy: 0.3333
Epoch 3/50
9/9 [==============================] - 0s 3ms/step - loss: 1.2565 - accuracy: 0.3333
Epoch 4/50
9/9 [==============================] - 0s 3ms/step - loss: 1.1593 - accuracy: 0.3259
Epoch 5/50
9/9 [==============================] - 0s 2ms/step - loss: 1.1367 - accuracy: 0.2370
Epoch 6/50
9/9 [==============================] - 0s 2ms/step - loss: 1.1282 - accuracy: 0.1407
Epoch 7/50
9/9 [==============================] - 0s 2ms/step - loss: 1.1241 - accuracy: 0.1407
Epoch 8/50
9/9 [==============================] - 0s 3ms/step - loss: 1.1229 - accuracy: 0.1333
Epoch 9/50
9/9 [==============================] - 0s 3ms/step - loss: 1.1202 - accuracy: 0.1926
Epoch 10/50
9/9 [==============================] - 0s 3ms/step - loss: 1.1174 - accuracy: 0.1704
Epoch 11/50
9/9 [======================

### 1.2 Is your (deep) network better than SVM? Why or why not? (4 points)

Please write your answer here.